In [2]:
import pandas as pd
import numpy as np
import gc
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from torch.optim import Adam
import networkx as nx

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### GraphSAGE load

In [2]:
#load data
classes = pd.read_csv('Elliptic++Dataset/txs_classes.csv')
features = pd.read_csv('Elliptic++Dataset/txs_features.csv')
edges = pd.read_csv('Elliptic++Dataset/txs_edgelist.csv')

#Convert classes to numerical format
classes["class"] -= 1
features["class"] = classes["class"]
# class_map = {"unknown": 2, "licit": 0, "illicit": 1}
# classes["class"] = classes["class"].map(class_map)

# #extract IDs, features and labels
features = features.dropna()
x = torch.as_tensor(features.iloc[:, 1:].drop("class", axis=1).values.astype(np.float32))
y = torch.as_tensor(features["class"].values)

#edges to tensor
edge_index = torch.as_tensor(edges.values.T)

#create data object
data = Data(x=x, edge_index=edge_index, y=y).to(device)
print(data)

Data(x=[202804, 183], edge_index=[2, 234355], y=[202804])


In [6]:
class GraphSAGE(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
    super(GraphSAGE, self).__init__()
    self.conv1 = SAGEConv(in_channels, hidden_channels)
    self.conv2 = SAGEConv(hidden_channels, out_channels)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = self.conv2(x, edge_index)
    return F.log_softmax(x, dim=1)

### Setup GraphSAGE for Elliptic

In [4]:
from data_processing import *

data, OUT_DIM = load_create_ellipticpp(timestep=(1,5))

In [12]:
from tqdm import tqdm

#dataLoader for neighborhood sampling
# train_mask = torch.zeros(data.num_nodes,dtype=torch.bool)
# train_mask[:int(0.7*data.num_nodes)]= True
# data.train_mask = train_mask


model = GraphSAGE(in_channels=data.num_features, hidden_channels = 32, out_channels = OUT_DIM).to(device)
train_loader = NeighborLoader(data=data, num_neighbors = [10,5], batch_size = 1000)
# #define optimizer and loss function
optimizer = Adam(model.parameters(), lr= 0.005, weight_decay = 5e-4)
loss_fn = torch.nn.CrossEntropyLoss()

def train(iterator):
  model.train()
  total_loss = 0
    
  iterator = tqdm(train_loader, )
  for idx, batch in enumerate(train_loader):
    optimizer.zero_grad()
    out = model(batch.x, batch.edge_index)
    # print(f"Output Shape: {out.shape}, Label Shape: {batch.y.shape}")

    # if torch.isnan(out).any():
    #     print("warning:NaN detected in output")
    #     return total_loss
        
    mask = batch.train_mask
    loss = loss_fn(out[mask], batch.y[mask])
      
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    iterator.set_description(f"Batch {idx+1}{data.size()[0]//1000}")
  return total_loss 

iterator = tqdm(range(20), desc="")
for idx, epoch in enumerate(iterator):
  loss = train(iterator)
  print(f"Epoch {idx+1}/{20} - Loss: {loss:.4f}")

  5%|▌         | 1/20 [00:00<00:15,  1.21it/s]

Epoch 1/20 - Loss: 101540.5549


 10%|█         | 2/20 [00:01<00:15,  1.19it/s]

Epoch 2/20 - Loss: 48146.0492


 15%|█▌        | 3/20 [00:02<00:14,  1.21it/s]

...

Epoch 18/20 - Loss: 13.2690


 95%|█████████▌| 19/20 [00:16<00:00,  1.15it/s]

Epoch 19/20 - Loss: 11.8873


100%|██████████| 20/20 [00:17<00:00,  1.15it/s]


Epoch 20/20 - Loss: 11.8602
